In [ ]:
import os
from numba import guvectorize
import numpy as np
from math import sin,cos,sqrt,pi,floor,ceil,log,acos
from timeit import default_timer as timer
import pdb
from matplotlib import pyplot
import pylab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
global itv,wc,acc,itv,titv,r,rc,bhh,bht,btt,kbo,kbe,num,coord,velocity,accel,test,damp

saving_path= os.getcwd()
damp=np.float64(1.0)
num=450
itv=np.float64(0.05)
wc=np.float64(1.5)
titv=np.float64(0.005)
r=np.float64(1.0)
rc=np.float64(2**(1/6)*r)
bhh=np.float64(0.85)
bht=np.float64(0.95)
btt=np.float64(1.0)
kbo=np.float64(30.0)
kbe=np.float64(10.0)
coord=np.zeros([num,3,7,3],dtype=np.float64)
velocity=np.zeros([num,3,3],dtype=np.float64)
accel=np.zeros([num,3,3],dtype=np.float64)

@guvectorize(['void(float64[:,:,:,:], float64[:,:,:,:,:])'],
             '(n,m,o,p)->(n,m,o,n,p)',target='parallel')
def distance(A,B):
    for a in range(B.shape[0]): #lipid number start
        for b in range(B.shape[1]): #h,t1 or t2 start 
            for c in range(B.shape[2]):#surroundings which one for start
                for d in range(B.shape[3]): #ending lipid
                    for e in range(B.shape[4]):
                        B[a][b][c][d][e]=np.float64(sqrt((A[a][b][c][0]-A[d][e][0][0])**2+(A[a][b][c][1]-A[d][e][0][1])**2+(A[a][b][c][2]-A[d][e][0][2])**2))
                    
                    
                    
                    
                    
@guvectorize(['void(float64[:,:,:,:,:],float64[:,:,:,:], float64[:,:,:])'],
             '(n,m,o,p,q),(n,m,o,q)->(n,m,o)',target='parallel')
def potential(A,C,B): #distance, position output potential
    global rc,bhh,bht,btt,wc,kbo,kbe
    
    #initialize all values to 0
    for a in range(A.shape[0]):
        for b in range(A.shape[1]):
            for c in range(A.shape[2]): 
                B[a][b][c]=np.float64(0)
    

     #head and other lipids
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding position
            for d in list(list(range(0,a))+list(range(a+1,A.shape[0]))):
                rhh=A[a][0][c][d][0]
                rht1=A[a][0][c][d][1]
                rht2=A[a][0][c][d][2]
                
                if rhh <= rc:
                    B[a][0][c]+=np.float64(4*((bhh/rhh)**12-(bhh/rhh)**6+1/4))
                    
                if rht1 <= rc:
                    B[a][0][c]+=np.float64(4*((bht/rht1)**12-(bht/rht1)**6+1/4))
                    
                if rht2 <= rc:
                    B[a][0][c]+=np.float64(4*((bht/rht2)**12-(bht/rht2)**6+1/4))
                
#t1 and other lipids
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding position
            for d in list(list(range(0,a))+list(range(a+1,A.shape[0]))):
                rt1h=A[a][1][c][d][0]
                rt1t1=A[a][1][c][d][1]
                rt1t2=A[a][1][c][d][2]
                
                if rt1h <= rc:
                    B[a][1][c]+=np.float64(4*((bht/rt1h)**12-(bht/rt1h)**6+1/4))
                    
                if rt1t1 <= rc:
                    B[a][1][c]+=np.float64(-1 + 4*((btt/rt1t1)**12-(btt/rt1t1)**6+1/4))
                elif rt1t1 <= (rc+wc):
                    B[a][1][c]+= np.float64(-1*(cos((pi*(rt1t1-rc)/2/wc)))**2)
                    
                if rt1t2 <= rc:
                    B[a][1][c]+=np.float64(-1 + 4*((btt/rt1t2)**12-(btt/rt1t2)**6+1/4))
                elif rt1t2 <= (rc+wc):
                    B[a][1][c]+= np.float64(-1*(cos((pi*(rt1t2-rc)/2/wc)))**2)

    #t2 and other lipids
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding position
            for d in list(list(range(0,a))+list(range(a+1,A.shape[0]))):
                rt2h=A[a][2][c][d][0]
                rt2t1=A[a][2][c][d][1]
                rt2t2=A[a][2][c][d][2]
                if rt2h <= rc:
                    B[a][2][c]+=np.float64(4*((bht/rt2h)**12-(bht/rt2h)**6+1/4))
                    
                if rt2t1 <= rc:
                    B[a][2][c]+=np.float64(-1 + 4*((btt/rt2t1)**12-(btt/rt2t1)**6+1/4))
                elif rt2t1 <= (rc+wc):
                    B[a][2][c]+= np.float64(-1*(cos((pi*(rt2t1-rc)/2/wc)))**2)
                    
                if rt2t2 <= rc:
                    B[a][2][c]+=np.float64(-1 + 4*((btt/rt2t2)**12-(btt/rt2t2)**6+1/4))
                elif rt2t2 <= (rc+wc):
                    B[a][2][c]+= np.float64(-1*(cos((pi*(rt2t2-rc)/2/wc)))**2)
    #potential due to bond for h
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding number
            rht1b=A[a][0][c][a][1]-2.0
            y1=(C[a][0][c]-C[a][1][0])
            y2=(C[a][2][0]-C[a][1][0])
            y=y1[0]*y2[0]+y1[1]*y2[1]+y1[2]*y2[2]
            x=(y/(A[a][0][c][a][1])/(A[a][1][0][a][2]))
            if x<-1:
                x=-1
            rhs=np.float64(acos(x))
            B[a][0][c]+=np.float64((-0.5)*(kbo)*((1.5)**2)*log(1-(rht1b/1.5)**2) + (0.5)*(kbe)*(rhs-4)**2)
            
    #potential due to bond for t1
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding number
            rt1hb=A[a][1][c][a][0]-2.0
            rt1t2b=A[a][1][c][a][2]-2.0
            y1=(C[a][0][0]-C[a][1][c])
            y2=(C[a][2][0]-C[a][1][c])
            y=y1[0]*y2[0]+y1[1]*y2[1]+y1[2]*y2[2]
            x=(y/(A[a][1][c][a][1])/(A[a][1][c][a][2]))
            if x<-1:
                x=-1
            rt1s=np.float64(acos(x))
            B[a][1][c]+=np.float64((-0.5)*(kbo)*((1.5)**2)*log(1-(rt1t2b/1.5)**2) + (0.5)*(kbe)*(rt1s-4)**2 + (-0.5)*(kbo)*((1.5)**2)*log(1-(rt1hb/1.5)**2))
            
    #potential due to bond for t2
    for a in range(A.shape[0]): #lipid number
        for c in range(1,A.shape[2]): #surrounding number
            rt2t1b=A[a][2][c][a][1]-2.0
            y1=(C[a][0][0]-C[a][1][0])
            y2=(C[a][2][c]-C[a][1][0])
            y=y1[0]*y2[0]+y1[1]*y2[1]+y1[2]*y2[2]
            x=(y/(A[a][0][0][a][1])/(A[a][2][c][a][1]))
            if x<-1:
                x=-1
            rt2s=np.float64(acos(x))
            B[a][2][c]+=np.float64((-0.5)*(kbo)*((1.5)**2)*log(1-(rt2t1b/1.5)**2) + (0.5)*(kbe)*(rt2s-4)**2)            

def update():
    global coord,velocity,accel,itv,titv,num
    dist=distance(coord)
    pot=potential(dist,coord)
    for a in range(num):
        for b in range(3):
            accel[a][b][0]= (pot[a][b][2]-pot[a][b][1])/2/itv-1*velocity[a][b][0]
            accel[a][b][1]= (pot[a][b][4]-pot[a][b][3])/2/itv-1*velocity[a][b][1]
            accel[a][b][2]= (pot[a][b][6]-pot[a][b][5])/2/itv-1*velocity[a][b][2]
            
    s=np.array(np.random.normal(0, 1,[num,3,3]),dtype=np.float64) #random values squared average of 1
    accel += s
    velocity += accel*titv 
    for a in range(num):
        for b in range(3):
            coord[a][b][0]+= velocity[a][b]*titv+0.5*(accel[a][b])*titv**2
    
    for a in range(0,num):
        for d in range(0,3):
            coord[a][d][1]=coord[a][d][0]+[itv,0,0]
            coord[a][d][2]=coord[a][d][0]-[itv,0,0]
            coord[a][d][3]=coord[a][d][0]+[0,itv,0]
            coord[a][d][4]=coord[a][d][0]-[0,itv,0]
            coord[a][d][5]=coord[a][d][0]+[0,0,itv]
            coord[a][d][6]=coord[a][d][0]-[0,0,itv]

    accel=np.zeros([num,3,3],dtype=np.float64)
    
    return


def main():
    for ii in range(0,10000,1):
        fig = pylab.figure()
        ax = Axes3D(fig)
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')
        ax.set_xlim3d(-5,50)
        ax.set_ylim3d(-5,50)
        ax.set_zlim3d(-5,50)
        ax.view_init(elev=0, azim=45)


        for i in range(num):
            ax.scatter3D(coord[i][0][0][0],coord[i][0][0][1],coord[i][0][0][2],s=50, c='r',marker="o")
            ax.scatter3D(coord[i][1][0][0],coord[i][1][0][1],coord[i][1][0][2],s=20, c='g',marker="o")
            ax.scatter3D(coord[i][2][0][0],coord[i][2][0][1],coord[i][2][0][2],s=20, c='b',marker="o")

        ax.text(5,5,45, "The w_c, itv and titv is " + str(wc) +", " + str(itv) +", " + str(titv), fontsize=7)
        ax.text(5,5,42, "The bhh, btt and bht is " + str(bhh) +", "+ str(btt) +", " + str(bht), fontsize=7)
        ax.text(5,5,39, "The kbo and kbe is " + str(kbo)+", "+ str(kbe), fontsize=7)
        ax.text(5,5,36, "The number of lipids is "+ str(num), fontsize=7)
        #fig.set_size_inches(10, 10)

        pyplot.savefig(saving_path+("\\graphs\\plot%d.png" %ii),format='png', dpi=150)
        plt.close()

        for iii in range(0,100):
            update()
    return
            

#lipid initialization
a=0
for b in np.arange(10,40,2):
    for c in np.arange(10,40,2):
        k=1
        for d in np.arange(0,3):
            coord[a][d][0]=np.array([b,c,k],dtype=np.float64)
            k+=2
        a+=1
for b in np.arange(10,40,2):
    for c in np.arange(10,40,2):
        k=10.5
        for d in np.arange(0,3):
            coord[a][d][0]=np.array([b,c,k],dtype=np.float64)
            k-=2
        a+=1

for a in range(0,num):
    for d in range(0,3):
        coord[a][d][1]=coord[a][d][0]+[itv,0,0]
        coord[a][d][2]=coord[a][d][0]-[itv,0,0]
        coord[a][d][3]=coord[a][d][0]+[0,itv,0]
        coord[a][d][4]=coord[a][d][0]-[0,itv,0]
        coord[a][d][5]=coord[a][d][0]+[0,0,itv]
        coord[a][d][6]=coord[a][d][0]-[0,0,itv]

    
if __name__ == "__main__":
    main()